In [1]:
import pandas as pd
# import libraries and packages
import numpy as np
import pandas as pd
import re
import time
from nltk import word_tokenize
import string
from tqdm import tqdm
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from textblob import TextBlob
data_train=pd.read_csv("/content/drive/MyDrive/fake_news_classification_mal_train.csv")


In [2]:
import pandas as pd

# Use pd.read_excel instead of pd.read_csv for Excel files
data_test = pd.read_excel("/content/drive/MyDrive/fake_news_classification_mal_test.xlsx", engine='openpyxl')  # Remove encoding argument

In [3]:
data_test.head()

,S.no,News
0,FAKE_MAL_TE_0001,കേരളത്തില്‍ പുരുഷന്മാര്‍ക്ക് രണ്ട് ഭാര്യമാര്‍ ...
1,FAKE_MAL_TE_0002,പാർട്ടിയുടെ കൊടിക്ക് മഹത്വം ഉണ്ടെന്നും സംരംഭങ്...
2,FAKE_MAL_TE_0003,നവകേരള സദസ്സ്: കാട്ടാക്കട ക്രിസ്ത്യൻ കോളേജ് കവ...
3,FAKE_MAL_TE_0004,ശബരിമലയില്‍ അയ്യപ്പ ഭക്തന്‍റെ തല പോലീസ് അടിച്ച...
4,FAKE_MAL_TE_0005,ബൈക്കുകള്‍ സ്വന്തം ജില്ലയില്‍ മാത്രം ഉപയോഗിക്ക...


In [4]:
data_train.head()

,ID,News,Label
0,FAKE_MAL_TR_0001,കേള്‍വി തകരാറുള്ള കുട്ടികള്‍ക്ക് നടത്തുന്ന സൗജ...,FALSE
1,FAKE_MAL_TR_0002,ചന്ദ്രയാന് കേരള മുഖ്യമന്ത്രി പിണറായി വിജയൻ മാത...,FALSE
2,FAKE_MAL_TR_0003,പിണറായി വിജയന്‍ സര്‍ക്കാര്‍ നിര്‍മിച്ച കേരളത്ത...,FALSE
3,FAKE_MAL_TR_0004,വിഴിഞ്ഞത്ത് തീരദേശവാസികള്‍ ആക്രമിച്ചപ്പോള്‍ മു...,FALSE
4,FAKE_MAL_TR_0005,കുരിശിന് മുന്നില്‍ കൈകൂപ്പി നില്‍ക്കുന്ന പിണറാ...,FALSE


In [5]:
data_train['Label'].value_counts()

,count
Label,
FALSE,1220
MOSTLY FALSE,295
FALSE,166
HALF TRUE,162
PARTLY FALSE,57


**Preprocessing**

In [6]:
def clean_text(text):
    stop_words=open(r'/content/drive/MyDrive/icon_task_2020/malayalam_stopwords.txt', encoding='utf=8').read().split('\n')
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))
    text = "".join([word for word in text if word not in string.punctuation])
    text = "".join([word for word in text if not word.isdigit()])
    tokens = re.split('\s+',text)
    text = " ".join(word for word in tokens if word not in stop_words)          # here english words may lemmatize so kept lematization
    return text

In [7]:
data_train['clean'] = data_train['News'].apply(lambda x:clean_text(x))
data_test['clean'] = data_test['News'].apply(lambda x:clean_text(x))

In [8]:
data_test.head()
X_test_withoutL=data_test['clean']

In [29]:



from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocabulary_size = 30000  # Define vocabulary_size before using it
tokenizer = Tokenizer(num_words=vocabulary_size)
tokenizer.fit_on_texts(data_train['clean'])
sequences = tokenizer.texts_to_sequences(X_test_withoutL)
X_test_withoutL1 = pad_sequences(sequences, maxlen=100)

In [30]:
sequences = tokenizer.texts_to_sequences(X_test_withoutL)
X_test_withoutL1 = pad_sequences(sequences, maxlen=100)

**Train data split**

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data_train['clean'] , data_train['Label'] , test_size=0.2, random_state=42)

In [14]:
!pip install tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocabulary_size = 30000  #
tokenizer = Tokenizer(num_words= vocabulary_size) # Now Tokenizer is defined from keras.preprocessing.text
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train1 = pad_sequences(sequences, maxlen=100)
sequences = tokenizer.texts_to_sequences(X_test)
X_test1 = pad_sequences(sequences, maxlen=100)



In [15]:

y = pd.get_dummies(y_train).values

y_train1 = pd.get_dummies(y_train).values
y_test1 = pd.get_dummies(y_test).values
diz_label1 = {}
for i,label in enumerate(data_train.Label.factorize()[1]):
    diz_label1[i] = label

diz_label1

{0: 'FALSE     ',
 1: 'HALF TRUE',
 2: 'FALSE',
 3: 'MOSTLY FALSE',
 4: 'PARTLY FALSE'}

In [16]:
embeddings_index = {}
with open("/content/drive/MyDrive/icon_task_2020/wiki.ml.vec",encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

<ipython-input-16-180427045892>:5: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  coefs = np.fromstring(coefs, "f", sep=" ")


Found 189665 word vectors.


In [17]:
VOCAB_SIZE = len(tokenizer.word_index) + 2
EMBEDDING_DIM = 300
hits = 0
misses =0

# Prepare embedding matrix
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        misses += 1

print("Converted %d words (%d misses)" % (hits, misses))

Converted 4662 words (3557 misses)


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GRU, Dense, Dropout

**GRU**

In [20]:
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], input_length=100, trainable=False))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(GRU(128))
model.add(Dropout(0.2))  # Add dropout for regularization
model.add(Dense(5, activation='softmax'))  # Assuming binary classification

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.fit(X_train1, y_train1, epochs=50, batch_size=64, validation_data=(X_test1, y_test1))

Epoch 1/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - accuracy: 0.5688 - loss: 1.3200 - val_accuracy: 0.6842 - val_loss: 1.0458
Epoch 2/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6346 - loss: 1.0470 - val_accuracy: 0.6816 - val_loss: 0.9958
Epoch 3/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6625 - loss: 0.8911 - val_accuracy: 0.6711 - val_loss: 0.9970
Epoch 4/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7723 - loss: 0.6446 - val_accuracy: 0.6737 - val_loss: 1.0516
Epoch 5/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9084 - loss: 0.3290 - val_accuracy: 0.6789 - val_loss: 1.1572
Epoch 6/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9788 - loss: 0.1326 - val_accuracy: 0.6658 - val_loss: 1.3083
Epoch 7/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9898 - loss: 0.0677 - val_accuracy: 0.6447 - val_loss: 1.4605
Epoch 8/50
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9940 - loss: 0.0438 - val_accuracy: 0.6526 - v

In [24]:
pred3 = model.predict(X_test1)
# y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test1, axis=1)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [ ]:

from sklearn.metrics import classification_report
print(classification_report(y_true_classes, y_pred_classes))

              precision    recall  f1-score   support

           0       0.73      0.88      0.80       260
           1       0.28      0.17      0.21        29
           2       0.38      0.13      0.19        23
           3       0.35      0.24      0.29        58
           4       1.00      0.10      0.18        10

    accuracy                           0.67       380
   macro avg       0.55      0.31      0.34       380
weighted avg       0.63      0.67      0.63       380



**LSTM**

In [25]:


model1 = Sequential()
model1.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True))
model1.add(LSTM(128, return_sequences=True))  # LSTM layer with return_sequences=True
model1.add(Dropout(0.2))
model1.add(Flatten())
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(5, activation='softmax'))

In [26]:
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model1.fit(X_train1, y_train1, epochs=50, batch_size=64, validation_data=(X_test1, y_test1))

In [28]:
pred1 = model1.predict(X_test1)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [ ]:
print(classification_report(np.argmax(y_test1,axis=1),np.argmax(pred1,axis=1)))

              precision    recall  f1-score   support

           0       0.75      0.70      0.72       260
           1       0.26      0.21      0.23        29
           2       0.19      0.13      0.15        23
           3       0.20      0.31      0.24        58
           4       0.17      0.10      0.12        10

    accuracy                           0.55       380
   macro avg       0.31      0.29      0.29       380
weighted avg       0.58      0.55      0.56       380



**LSTM+GRU**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping  # Import EarlyStopping

model2 = Sequential()
model2.add(Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], trainable=True))
model2.add(LSTM(128, return_sequences=True))  #LSTM layer with return_sequences=True
model2.add(GRU(64))  #GRU layer after LSTM
model2.add(Dropout(0.2))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(5, activation='softmax'))  #Assuming 5 output classes

model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Now you can use EarlyStopping in your fit method
model2_history = model2.fit(X_train1, y_train1, epochs=40, batch_size=225,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.4066 - loss: 1.4980 - val_accuracy: 0.6250 - val_loss: 1.1992
Epoch 2/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.6457 - loss: 1.1486 - val_accuracy: 0.6250 - val_loss: 1.1677
Epoch 3/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 988ms/step - accuracy: 0.6205 - loss: 1.1928 - val_accuracy: 0.6250 - val_loss: 1.1469
Epoch 4/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 949ms/step - accuracy: 0.6341 - loss: 1.1455 - val_accuracy: 0.6250 - val_loss: 1.1422
Epoch 5/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.6269 - loss: 1.1201 - val_accuracy: 0.6250 - val_loss: 1.1155
Epoch 6/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.6444 - loss: 1.0533 - val_accuracy: 0.6250 - val_loss: 1.0938
Epoch 7/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 950ms/step - accuracy: 0.6496 - loss: 0.9655 - val_accuracy: 0.6382 - val_loss: 1.0858
Epoch 8/40
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.6947 - loss: 0.9037 - val_accuracy: 0.6382 - val_loss: 1.1637
Ep

In [ ]:
pred2 = model2.predict(X_test1)


12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step


**Submission File**

In [ ]:
pred2 = model2.predict(X_test_withoutL1)
pred1 = model1.predict(X_test_withoutL1)
pred3 = model.predict(X_test_withoutL1)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 192ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step


In [ ]:
decoded_pred2 = [diz_label1[np.argmax(index)] for index in pred2]  # Use np.argmax to get the index with the highest probability
submission_df = pd.DataFrame({'id': data_test['S.no'], 'label': decoded_pred2})
submission_df.to_csv('YenCS_task2_run1.tsv', index=False)

In [ ]:
decoded_pred1 = [diz_label1[np.argmax(index)] for index in pred1]  # Use np.argmax to get the index with the highest probability
submission_df = pd.DataFrame({'id': data_test['S.no'], 'label': decoded_pred1})
submission_df.to_csv('YenCS_task2_run4.tsv', index=False)

In [ ]:
decoded_pred3 = [diz_label1[np.argmax(index)] for index in pred3]  # Use np.argmax to get the index with the highest probability
submission_df = pd.DataFrame({'id': data_test['S.no'], 'label': decoded_pred3})
submission_df.to_csv('YenCS_task2_run5.tsv', index=False)